In [71]:
#Requires PySpark, installation information can be found here: https://www.dataquest.io/blog/installing-pyspark/
#import PySpark as ps 
import feedparser as fp, pandas as pd
from bs4 import BeautifulSoup 

#Get the News Feed and Slice it to only include the news items
feed = fp.parse('http://feeds.huffingtonpost.com/c/35496/f/677097/index.rss')
feed_items = feed['items']

In [72]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the cleaned Dataframe    
news_df = dataframe_cleaner(raw_news_dataframe)
news_df.head()

,author,id,title,summary
0,Daryl Deino,http://www.huffingtonpost.com/daryl-deino/senn...,Sennheiser M2 Wireless Headphones Are Best Blu...,Photo by Daryl Deino The Sennheiser M2 Wirel...
1,Andy Plesser,http://www.huffingtonpost.com/andy-plesser/vid...,(VIDEO) OpenX Hoping To Optimize The Use Of Mu...,The increased popularity of header bidding has...
2,Damon Beres,http://www.huffingtonpost.com/2016/03/29/distr...,Sending A Text Behind The Wheel Is Like Drivin...,If you look at your phone for just four secon...
3,Andy Plesser,http://www.huffingtonpost.com/andy-plesser/vid...,(VIDEO) The Trade Desk Rolling Out Programmati...,MIAMI - Demand-side platform The Trade Desk be...
4,Erin Schumaker,http://www.huffingtonpost.com/2016/03/29/telem...,The Tech Innovation That Might Make It Easier ...,(Reuters Health) - Women in the U.S. without ...


In [101]:
###############################Import, Clean and DataFrame the Sentiment Dictionary################################

#Import the Sentiment Dictionary 
f = open('sentiment_dictionary.tff','r')
raw_sentiment_dictionary = f.read()


#Create a Totally Cleaned Sentiment Dataframe
def sentiment_dataframe_maker(file):
    
    #Call another function to turn the file into a raw dataframe
    raw_df = dictionary_to_dataframe(file)
    raw_df = raw_df.dropna()
    #Clean up the strings within the cells
    replace_list = ['type','word','position','stemmed','polarity']
    
    for x in replace_list:
        raw_df[x] = list(map(lambda x: x.split('=')[-1],raw_df[x]))
        
    #Convert Type and Polarity to numerical values 
    
    type_unique = dict.fromkeys(raw_df['type'].unique())
    type_unique['weaksubj'] = .5
    type_unique['strongsubj'] = 1
    
    for key,value in type_unique.items():
        raw_df['type'] = raw_df['type'].replace(key,value)  
    
    polarity_unique = dict.fromkeys(raw_df['polarity'].unique())
    polarity_unique['negative'] = int(-1)
    polarity_unique['positive'] = int(1)
    polarity_unique['neutral'] = 0
    polarity_unique['both'] = None
    polarity_unique['m'] = None
    
    for key,value in polarity_unique.items():
        raw_df['polarity'] = raw_df['polarity'].replace(key,value)
    
    raw_df = raw_df.dropna()
    
    return raw_df
    
    
    
#Cleanse the Dictionary and create a Raw Sentiment DataFrame
def dictionary_to_dataframe(dictionary):
    
    being_cleaned_list = dictionary.split('\n')
    being_cleaned_list = [x.split(',') for x in being_cleaned_list]
    being_cleaned_list = list(map(lambda x: x[0].split(" "),being_cleaned_list))
    being_cleaned_df = pd.DataFrame(being_cleaned_list)
    being_cleaned_df = being_cleaned_df[[0,2,3,4,5]]
    being_cleaned_df.columns = ['type','word','position','stemmed','polarity']
    
    return being_cleaned_df
    

    
sentiment_df = sentiment_dataframe_maker(raw_sentiment_dictionary)
sentiment_df.head(1)

8222
8222
[ 0.5  1. ] [-1  1  0]


,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1
